<a href="https://colab.research.google.com/github/antoinebachand/Deep-Learnig-/blob/main/Float_1target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Network for Prediction of Hydrogen Thermodynamics**
# **Float with 1 Targets**
### Author: Antoine Bachand (antoinebachand@outlook.com)

The training data set is obtained through a model adapted from Kushnir et al. (2012) for hydrogen storage. DOI: 10.1016/j.ijheatmasstransfer.2012.05.055 

In [31]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as functional
import torch.nn as nn

In [32]:
#load the data
df = pd.read_csv('data_10000.csv')
df.columns = ['Injection Temperature (K)', 'Intitial Temperature (K)', 'Total Mass (kg)', 'Injection Time (s)', 'Max Pressure (MPa)','Max Temperature (Celsius)']

The inputs to the neural network are:
- Initial rock temperature (K)
- Injection temperature (K)
- Total injection mass (kg)
- Injection time (s) 

and the outputs are as follows:
- Maximum pressure ( MPa )
- Maximum temperature ( °C ) 

In [33]:
df.describe()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Max Pressure (MPa),Max Temperature (Celsius)
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,373.359956,292.831066,1251.091858,13534.993797,70.186064,87.375692
std,57.798521,11.607488,431.115254,5781.543880,61.757836,166.266974
min,273.028357,273.005582,500.074091,3600.220147,16.635890,9.773542
25%,323.171228,282.742639,877.214602,8496.583859,37.319443,52.352185
50%,373.583463,292.672869,1255.380370,13463.256415,61.838304,70.354742
75%,423.437581,302.876639,1626.478623,18582.071768,91.863710,93.249890
max,472.990125,312.998830,1999.955495,23599.455440,2369.358069,8172.750350


# Modification

Here we simplify the dataset by considering a single target or output.  Therefore we subtract the Max Temperature column from the data frame

In [36]:
Round_Max_Pressure = []

for i in df['Max Pressure (MPa)']:
  Round_Max_Pressure.append(math.ceil(i))

df['Round Max Pressure (MPa)'] = Round_Max_Pressure

df_drop = df.drop(['Max Temperature (Celsius)','Max Pressure (MPa)'],axis=1)
df_drop2 = df_drop.drop(['Round Max Pressure (MPa)'],axis=1)
df_drop.head()


,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Round Max Pressure (MPa)
0,282.469062,300.763007,1869.140142,13607.354450,111
1,399.261432,306.867055,611.049819,14247.207020,25
2,332.949980,308.329176,1435.526958,5114.476186,80
3,398.578465,303.391355,1600.185040,15412.538030,93
4,329.299607,304.099858,871.039883,8019.916604,38


In [67]:
df_drop.describe()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Round Max Pressure (MPa)
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,373.359956,292.831066,1251.091858,13534.993797,70.682800
std,57.798521,11.607488,431.115254,5781.543880,61.756025
min,273.028357,273.005582,500.074091,3600.220147,17.000000
25%,323.171228,282.742639,877.214602,8496.583859,38.000000
50%,373.583463,292.672869,1255.380370,13463.256415,62.000000
75%,423.437581,302.876639,1626.478623,18582.071768,92.000000
max,472.990125,312.998830,1999.955495,23599.455440,2370.000000


Since our different features are on different ranges, a normalization is applied.

In [68]:
# Normalization
norm = MinMaxScaler().fit(df_drop2) 
nd_norm = norm.transform(df_drop2)
df_norm = pd.DataFrame(nd_norm)

df_norm.columns = ['Injection Temperature (K)', 'Intitial Temperature (K)', 'Total Mass (kg)', 'Injection Time (s)']
df_norm['Round Max Pressure (MPa)'] = df_drop['Round Max Pressure (MPa)']

In [69]:
df_norm.head()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Round Max Pressure (MPa)
0,0.047213,0.694053,0.912783,0.500376,111
1,0.631286,0.846680,0.073990,0.532370,25
2,0.299665,0.883239,0.623685,0.075716,80
3,0.627871,0.759773,0.733465,0.590638,93
4,0.281410,0.777488,0.247330,0.220993,38


#Preparing data for training/testing

Here we split the data between training (80%) and testing (20%) in order to test the neural network with unseen data

In [70]:
# The columns are split between independent (Input) and dependent (Output)
columns = df_norm.columns.values.tolist()
# The independent ones correspond to the last two columns of the data set. 
independent = columns[:-1]  
dependent = columns[-1]     
# The test data size is set at 20%. 
X_train, X_test, y_train, y_test = train_test_split(df_norm[independent],df_norm[dependent], test_size=0.2)

Now we will create the tensors. They correspond to the structure used to store information/data in machine learning. 

Also, since the stochastic gradient descent computes the gradient on a subset of the training data, as opposed to the entire training dataset, we need to specify a batch size. Therefore, to avoid generalization, we will use a small batch size of 50. 

In [71]:
BATCH_SIZE = 50
# Tensor for the training data and the value type is float 
X = torch.tensor(X_train.values, dtype=torch.float)
y = torch.tensor(y_train.values, dtype=torch.int64)

train_ds = torch.utils.data.TensorDataset(X,y)

# Tensor for the testing data and the value type is float
X = torch.tensor(X_test.values, dtype=torch.float)
y = torch.tensor(y_test.values, dtype=torch.int64)

test_ds = torch.utils.data.TensorDataset(X,y)

# We shuffle the training dataset
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# Network architecture
The architecture of the neural network consists of 3 hidden layers with 50 neurons. The activation function used is ReLU except for the output layer we use the sigmoid function since the output is binary. 

In [73]:
class MyANN(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(4, 50) 
    self.av1 = nn.ReLU()   
    self.fc2 = nn.Linear(50, 50)    
    self.av2 = nn.ReLU()
    self.fc3 = nn.Linear(50, 50) 
    self.av3 = nn.ReLU() 
    self.fc4 = nn.Linear(50, 50) 
    self.av4 = nn.ReLU()
    self.fc5 = nn.Linear(50, 2371) 
    self.av5 = nn.Sigmoid()  
    return


  def forward(self, x):
    x = self.fc1(x)
    x = self.av1(x)
    x = self.fc2(x)
    x = self.av2(x)
    x = self.fc3(x)
    x = self.av3(x)
    x = self.fc4(x)
    x = self.av4(x)
    x = self.fc5(x)
    x = self.av5(x)
    return nn.functional.log_softmax(x, dim=1)

net = MyANN()
print(net)

MyANN(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (av1): ReLU()
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (av2): ReLU()
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (av3): ReLU()
  (fc4): Linear(in_features=50, out_features=50, bias=True)
  (av4): ReLU()
  (fc5): Linear(in_features=50, out_features=2371, bias=True)
  (av5): Sigmoid()
)


# Training
At the training level the optimizer used is Adam. The learning rate was initially set to 0.001 and the number of epochs to 200.

In [74]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for epocs in range(100):
  for data in train_dl:
    X, y = data
    optimizer.zero_grad() # clear gradient information.
    output = net(X.view(-1, 4))
    loss = nn.functional.nll_loss(output, y)
    loss.backward() # do back-propagation step
  optimizer.step() # tell optimizer that you finished batch/iteration.
  print(loss.data)

tensor(7.7722)
tensor(7.7749)
tensor(7.7734)
tensor(7.7721)
tensor(7.7721)
tensor(7.7697)
tensor(7.7648)
tensor(7.7647)
tensor(7.7608)
tensor(7.7661)
tensor(7.7650)
tensor(7.7568)
tensor(7.7649)
tensor(7.7578)
tensor(7.7573)
tensor(7.7549)
tensor(7.7436)
tensor(7.7475)
tensor(7.7485)
tensor(7.7470)
tensor(7.7416)
tensor(7.7342)
tensor(7.7284)
tensor(7.7285)
tensor(7.7174)
tensor(7.7050)
tensor(7.7015)
tensor(7.7107)
tensor(7.6887)
tensor(7.6847)
tensor(7.6527)
tensor(7.6572)
tensor(7.6536)
tensor(7.6150)
tensor(7.6092)
tensor(7.5790)
tensor(7.5529)
tensor(7.5428)
tensor(7.5182)
tensor(7.5026)
tensor(7.4320)
tensor(7.4350)
tensor(7.4293)
tensor(7.3612)
tensor(7.3296)
tensor(7.3077)
tensor(7.2616)
tensor(7.3009)
tensor(7.2138)
tensor(7.1419)
tensor(7.1230)
tensor(7.1708)
tensor(7.1029)
tensor(7.0142)
tensor(7.0243)
tensor(7.0036)
tensor(6.9921)
tensor(6.9980)
tensor(6.9828)
tensor(6.9988)
tensor(6.9488)
tensor(6.9513)
tensor(6.9716)
tensor(6.9161)
tensor(6.9400)
tensor(6.8838)
tensor(6.9

In [75]:
total = 0
correct = 0
with torch.no_grad(): 
  for data in test_dl:
    X, y = data
    output = net(X.view(-1, 4))    
    for idx, val in enumerate(output):
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print('Accuracy:', round(correct/total, 3))

Accuracy: 0.017
